# ReadNAnalyzrBot

## Installing and Importing dependencies

In [84]:
# Instal the required modules:
#pip install textblob
#pip install beautifulsoup4
#pip install nltk

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import os
import chardet
import string
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from textblob import TextBlob

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load input Excel file that contains URL of the sights you need to analyze

In [184]:
input_file_location = r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\Input.xlsx'
input_file = pd.read_excel(input_file_location)
input_file

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


## Extracting texts

In [172]:
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extracting article title
    title_element_1 = soup.find('h1', class_='entry-title')
    title_element_2 = soup.find('h1', class_='tdb-title-text')

    # Use the title from the first found element (non-empty)
    title = title_element_1.text.strip() if title_element_1 else title_element_2.text.strip() if title_element_2 else ""
    
    # Extracting article text 
    article_text_element_1 = soup.find(class_="td-post-content tagdiv-type")

    if article_text_element_1:
        # Ignore text inside "wp-block-preformatted" class
        preformatted_elements = article_text_element_1.find_all(class_="wp-block-preformatted")
        for preformatted_element in preformatted_elements:
            preformatted_element.decompose()  # Remove the element and its content
        
        # Extract text after removing "wp-block-preformatted" elements
        article_text = article_text_element_1.text.strip()
    else:
        # If not found, try to find text inside <p>, <li>, and <u> tags in "tdb-block-inner td-fix-index"
        tdb_block_inner_elements = soup.find_all(class_="tdb-block-inner td-fix-index")
        combined_texts = []

        for tdb_block_inner in tdb_block_inner_elements:
            # Extract text from <p>, <li>, and <u> tags
            p_elements = tdb_block_inner.find_all(['p', 'li', 'u'])
            combined_texts.extend([element.text.strip() for element in p_elements if element.text.strip()])

        article_text = ' '.join(combined_texts)

    article_text = re.sub(r'\n', '', article_text)

    
    return title, article_text



#"td-post-content tagdiv-type"
#"tdb-block-inner td-fix-index"
#"tdb-title-text"

In [173]:
extract_article_text(r'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/')

('How will COVID-19 affect the world of work?',
 'As business close to help prevent transmission of COVID-19, financial concerns and job losses are one of the first human impacts of the virus; Not knowing how this pandemic will play out also affects our economic, physical and mental well-being; Despite this fear, businesses and communities in many regions have shown a more altruistic response in the face of crisis – actions which could help countries preparing for COVID-19. COVID-19 is in decline in China. There are now more new cases every day in Europe than there were in China at the epidemic’s peak and Italy has surpassed it as the country with the most deaths from the virus It took 67 days to reach the first 100,000 confirmed cases worldwide, 11 days for this to increase to 200,000and just four to reach 300,000 confirmed cases – a figure now exceeded. In recent weeks, we have seen the significant economic impact of the coronavirus on financial markets and vulnerable industries such

## Creating txt files out of extracted text content

In [140]:
for index, row in input_file.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    # Extract article text and title from the URL
    title, article_text = extract_article_text(url)
    
    # Save the extracted title and text in a text file
    output_folder = r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\Extracted Text files'
    with open(os.path.join(output_folder, f'{url_id}.txt'), 'w', encoding='utf-8') as file:
        file.write(f'{title}\n\n')
        file.write(f' {article_text}\n')
    

KeyboardInterrupt: 

## Functions to perform different text analysis

In [153]:
def perform_sentiment_analysis(text):
   
    full_text = text
    # Cleaning using Stop Words Lists
    stop_words_folder = r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\StopWords'
    stop_words = set()

    for filename in os.listdir(stop_words_folder):
        file_path = os.path.join(stop_words_folder, filename)
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            encoding_result = chardet.detect(raw_data)
            encoding = encoding_result['encoding']

        with open(file_path, 'r', encoding=encoding, errors='replace') as file:
            stop_words.update(file.read().split())

    # Tokenize the text
    tokens = word_tokenize(full_text.lower())
    cleaned_tokens = [word for word in tokens if word.lower() not in stop_words]

    print("Word count of the original article: ", len(tokens))
    print("Word count of the cleaned article: ", len(cleaned_tokens))

    # Creating a dictionary of Positive and Negative words
    positive_words = set(word for word in open(r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\MasterDictionary\positive-words.txt').read().split() if word not in stop_words)
    negative_words = set(word for word in open(r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\MasterDictionary/negative-words.txt').read().split() if word not in stop_words)

    # Extracting Derived variables
    positive_score = sum(1 for word in cleaned_tokens if word in positive_words)
    negative_score = sum(1 for word in cleaned_tokens if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(cleaned_tokens) + 0.000001)

    
    # Return the results
    return {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
    
        # Add other variables as needed
    }



In [59]:
url = r'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/'
title, article_text = extract_article_text(url)
sentiment_analysis_results = perform_sentiment_analysis(title, article_text)
print(analysis_results)

Word count of the original article:  1343
Word count of the cleaned article:  681
{'POSITIVE SCORE': 33, 'NEGATIVE SCORE': 5, 'POLARITY SCORE': 0.7368420858725767, 'SUBJECTIVITY SCORE': 0.05580029360381748, 'AVG SENTENCE LENGTH': 22.45, 'PERCENTAGE OF COMPLEX WORDS': 17.72151898734177}


In [60]:
    # Function to count syllables in a word
    def count_syllables(word):
        vowels = "aeiou"
        count = 0

        # Iterate through each letter in the word
        for char in word:
            # Count consecutive vowels
            if char.lower() in vowels:
                count += 1

        # Exclude words ending with "es" and "ed" from syllable count
        if word.lower().endswith(('es', 'ed')):
            count -= 1

        # Ensure a minimum count of 1
        return max(count, 1)



In [176]:
def perform_readability_analysis(text):
    
    full_text = text

    # Tokenize the text
    tokens = word_tokenize(full_text.lower())

    # let's calculate average sentence length
    sentences = sent_tokenize(full_text)
    avg_sentence_length = len(tokens) / len(sentences) if len(sentences) > 0 else 0

    # Count syllables for each word
    syllables_per_word = [count_syllables(word) for word in tokens]
    #print(syllables_per_word)

    # Calculate Percentage of Complex Words based on syllables
    complex_words = [word for word, syllables in zip(tokens, syllables_per_word) if syllables > 2]
    percentage_complex_words = (len(complex_words) / len(tokens)) * 100 if len(tokens) > 0 else 0
    
    # Calculate fog index
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)


   
    return {
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index
    }



In [179]:
url = r'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/'
title, article_text = extract_article_text(url)
full_text = f"{title} {article_text}"

readability_analysis_results = perform_readability_analysis(full_text)
print(readability_analysis_results)

{'AVG SENTENCE LENGTH': 22.166666666666668, 'PERCENTAGE OF COMPLEX WORDS': 17.593984962406015, 'FOG INDEX': 15.904260651629073}


In [182]:
def perform_basic_analysis(text):
    
    full_text = text

    # Tokenize the text
    tokens = word_tokenize(full_text.lower())
    
    #AVERAGE WORD PER SENTENCE
    sentences = sent_tokenize(full_text)
    avg_words_per_sentence = len(tokens)/len(sentences) if len(sentences) > 0 else 0
    syllables_per_word = [count_syllables(word) for word in tokens]
    complex_words = [word for word, syllables in zip(tokens, syllables_per_word) if syllables > 2]
    complex_wordcount = len(complex_words)
    
    stop_words2 = set(stopwords.words('english'))
    cleaned_tokens2 = [word for word in tokens if word.lower() not in stop_words2 and word not in string.punctuation]
    word_count = len(cleaned_tokens2)
    syllables_per_word = [count_syllables(word) for word in tokens]
    average_syllable_count = sum(syllables_per_word) / len(syllables_per_word) if len(syllables_per_word) > 0 else 0
    
    # Define the list of personal pronouns
    personal_pronouns = ["I", "we", "my", "ours", "us"]

    # Use regex to find counts of personal pronouns
    total_pronoun_count = sum(len(re.findall(r'\b' + pronoun + r'\b', full_text, flags=re.IGNORECASE)) for pronoun in personal_pronouns)
 
    # Special care to exclude the country name "US"
    total_pronoun_count -= len(re.findall(r'\bUS\b', full_text, flags=re.IGNORECASE))

    
    total_characters = sum(len(word) for word in tokens)

    # Calculate the total number of words
    total_words = len(tokens)

    # Calculate the average word length
    average_word_length = total_characters / total_words if total_words > 0 else 0

    return {
        'AVERAGE WORD PER SENTENCE': avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_wordcount,
        'WORD COUNT': word_count,
        'AVERAGE SYLLABLE COUNT':average_syllable_count,
        'PRONOUN COUNT':total_pronoun_count,
        'AVERAGE WORD LENGTH':average_word_length
    }



In [183]:
url = r'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/'
title, article_text = extract_article_text(url)
full_text = f"{title} {article_text}"

basic_analysis_results = perform_basic_analysis(full_text)
print(basic_analysis_results)

{'AVERAGE WORD PER SENTENCE': 22.166666666666668, 'COMPLEX WORD COUNT': 234, 'WORD COUNT': 626, 'AVERAGE SYLLABLE COUNT': 1.700751879699248, 'PRONOUN COUNT': 11, 'AVERAGE WORD LENGTH': 4.327067669172933}


## Writing the text analysis score on the output sheet

In [164]:
# Load the "Output Data Structure" Excel sheet into a DataFrame
output_excel_path = r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\Output Data Structure.xlsx'
df = pd.read_excel(output_excel_path)

# Folder path containing your text files
extracted_txt_folder_path = r'D:\DATA ANALYST PLAYGROUND\Blackcoffer\Extracted Text files'

# Iterate through rows of the DataFrame
for index, row in df.iterrows():
    # Extract URL_ID and construct file path
    url_id = row['URL_ID']
    txt_file_path = os.path.join(extracted_txt_folder_path, f'{url_id}.txt')
    
    # Check if the file exists
    if os.path.isfile(txt_file_path):
        # Read the text from the file
        with open(txt_file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            
        # Perform sentiment analysis
        sentiment_analysis_results = perform_sentiment_analysis(text)
        readability_analysis_results = perform_readability_analysis(text)
        basic_analysis_results = perform_basic_analysis(text)

        # Update the DataFrame with sentiment analysis results
        df.at[index, 'POSITIVE SCORE'] = sentiment_analysis_results['POSITIVE SCORE']
        df.at[index, 'NEGATIVE SCORE'] = sentiment_analysis_results['NEGATIVE SCORE']
        df.at[index, 'POLARITY SCORE'] = sentiment_analysis_results['POLARITY SCORE']
        df.at[index, 'SUBJECTIVITY SCORE'] = sentiment_analysis_results['SUBJECTIVITY SCORE']
        df.at[index, 'AVG SENTENCE LENGTH'] = readability_analysis_results['AVG SENTENCE LENGTH']
        df.at[index, 'PERCENTAGE OF COMPLEX WORDS'] = readability_analysis_results['PERCENTAGE OF COMPLEX WORDS']
        df.at[index, 'FOG INDEX'] = readability_analysis_results['FOG INDEX']
        df.at[index, 'AVG NUMBER OF WORDS PER SENTENCE'] = basic_analysis_results['AVERAGE WORD PER SENTENCE']
        df.at[index, 'COMPLEX WORD COUNT'] = basic_analysis_results['COMPLEX WORD COUNT']
        df.at[index, 'WORD COUNT'] = basic_analysis_results['WORD COUNT']
        df.at[index, 'SYLLABLE PER WORD'] = basic_analysis_results['AVERAGE SYLLABLE COUNT']
        df.at[index, 'PERSONAL PRONOUNS'] = basic_analysis_results['PRONOUN COUNT']
        df.at[index, 'AVG WORD LENGTH'] = basic_analysis_results['AVERAGE WORD LENGTH']

        # Add more columns as needed
# Write the updated DataFrame back to the Excel sheet
df.to_excel(output_excel_path, index=False)


Word count of the original article:  1330
Word count of the cleaned article:  670
Total number of cleaned words 670
Total number of sentences 60
Word count of the original article:  1668
Word count of the cleaned article:  988
Total number of cleaned words 988
Total number of sentences 64
Word count of the original article:  1194
Word count of the cleaned article:  755
Total number of cleaned words 755
Total number of sentences 46
Word count of the original article:  1181
Word count of the cleaned article:  733
Total number of cleaned words 733
Total number of sentences 38
Word count of the original article:  742
Word count of the cleaned article:  428
Total number of cleaned words 428
Total number of sentences 28
Word count of the original article:  2023
Word count of the cleaned article:  1274
Total number of cleaned words 1274
Total number of sentences 62
Word count of the original article:  1419
Word count of the cleaned article:  820
Total number of cleaned words 820
Total number 